In [9]:
import numpy as np
import pandas as pd
df = pd.read_csv("../data/成本价按日期单品合并_剔除后.csv",encoding='utf-8')
print(df)

           销售日期  102900005115762   102900005115779   102900005115786   \
0      2020/7/1              3.88              6.72              3.19   
1      2020/7/2              3.93              4.23              3.18   
2      2020/7/3              3.57              4.61              3.79   
3      2020/7/4              0.00              5.77              3.46   
4      2020/7/5              3.48              5.77              3.45   
...         ...               ...               ...               ...   
1090  2023/6/26              2.33              5.76              2.42   
1091  2023/6/27              2.34              5.74              2.47   
1092  2023/6/28              2.37              0.00              2.25   
1093  2023/6/29              2.37              5.68              2.16   
1094  2023/6/30              2.21              0.00              2.15   

      102900005115823   102900005115908   102900005115946   102900005115960   \
0                 7.03              4.60   

In [10]:
pl = pd.read_excel("../data/附件1.xlsx")
pl['单品编码'] = pl['单品编码'].astype(str)
pl = pl.set_index('单品编码')

In [11]:
# LSTM for international airline passengers problem with regression framing
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
tf.random.set_seed(7)

In [ ]:
predict = pd.DataFrame([])
RMSE = pd.DataFrame([])
for index in df:
	if index == '销售日期':
		continue
	# load the dataset
	name_CN = pl.loc[index[:-1]]['单品名称']
	print(name_CN)
	dataframe = df[['销售日期',index]]
	dataframe.set_index(["销售日期"], inplace=True)
	dataset = dataframe.values
	dataset = dataset.astype('float32')
	# normalize the dataset
	scaler = MinMaxScaler(feature_range=(0, 1))
	dataset = scaler.fit_transform(dataset)
	# split into train and test sets
	train_size = int(len(dataset) * 0.99)
	test_size = len(dataset) - train_size
	train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
	# reshape into X=t and Y=t+1
	look_back = 7
	trainX, trainY = create_dataset(train, look_back)
	testX, testY = create_dataset(test, look_back)
	# reshape input to be [samples, time steps, features]
	trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
	testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
	# create and fit the LSTM network
	model = Sequential()
	model.add(LSTM(30, input_shape=(1, look_back)))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	model.fit(trainX, trainY, epochs=30, batch_size=1, verbose=2)
	# make predictions
	trainPredict = model.predict(trainX)
	testPredict = model.predict(testX)
	# invert predictions
	trainPredict = scaler.inverse_transform(trainPredict)
	trainY = scaler.inverse_transform([trainY])
	testPredict = scaler.inverse_transform(testPredict)
	testY = scaler.inverse_transform([testY])
	# calculate root mean squared error
	trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
	print('Train Score: %.2f RMSE' % (trainScore))
	testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
	print('Test Score: %.2f RMSE' % (testScore))
	RMSE[name_CN] = [trainScore]
	print(trainScore)
	# shift train predictions for plotting
	trainPredictPlot = np.empty_like(dataset)
	trainPredictPlot[:, :] = np.nan
	trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
	# shift test predictions for plotting
	testPredictPlot = np.empty_like(dataset)
	testPredictPlot[:, :] = np.nan
	testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
	# plot baseline and predictions
	plt.plot(scaler.inverse_transform(dataset))
	plt.plot(trainPredictPlot)
	plt.plot(testPredictPlot)
	plt.show()

	testFinal = dataset[-look_back:]
	testFinal = [pd.DataFrame(testFinal).T.values.tolist()]
	testFinalPredict = model.predict(testFinal)
	testFinalPredict = scaler.inverse_transform(testFinalPredict)
	print(testFinalPredict)
	predict[name_CN] = testFinalPredict[0]
	print("--------------------------")

In [13]:
print(RMSE)
print(predict)

         苋菜      云南生菜       竹叶菜       上海青        菜心       木耳菜       大白菜  \
0  1.595777  1.242682  2.127125  0.961584  1.353451  1.349034  0.349218   

      云南油麦菜     红椒(1)  紫茄子(2)  ...     高瓜(2)  姜蒜小米椒组合装(小份)    双孢菇(盒)  \
0  0.962714  1.742241  1.0301  ...  2.468878       0.32644  0.394944   

     枝江青梗散花    鲜木耳(份)  青红杭椒组合装(份)    紫茄子(1)     红椒(2)  蟹味菇与白玉菇双拼(盒)    木耳菜(份)  
0  0.808137  0.244485    0.486981  0.519502  1.142464      0.546552  0.148935  

[1 rows x 61 columns]
         苋菜      云南生菜       竹叶菜       上海青        菜心       木耳菜       大白菜  \
0  1.779793  1.331318  1.928681  3.769756  1.624508  2.933631  2.478498   

      云南油麦菜     红椒(1)    紫茄子(2)  ...      高瓜(2)  姜蒜小米椒组合装(小份)    双孢菇(盒)  \
0  1.215161  0.363022  3.273593  ...  10.506674       1.95541  3.132378   

     枝江青梗散花  鲜木耳(份)  青红杭椒组合装(份)   紫茄子(1)     红椒(2)  蟹味菇与白玉菇双拼(盒)    木耳菜(份)  
0  5.352291   0.201    3.195488  0.84127  11.92913      2.308932  1.562065  

[1 rows x 61 columns]


In [14]:
RMSE.to_csv('../data/成本价RMSE.csv',sep=',',encoding='utf_8_sig')
predict.to_csv('../data/成本价7月1日预估.csv',sep=',',encoding='utf_8_sig')